These sets of codes were made on 7/5/2025. The objective was to estimate the drainage rates using the daily reference temperature differential data collected at the 60-cm depth from the Oklahoma Mesonet sites. The loaded files include matlab files containing the temperature differential data (from the MesoSoil database) and a csv file containing the Rosetta3-derived water retention, K0, and L parameters. The Rosetta3 parameters were used to estimate the fitted parameters K0 and L using Rosetta1 run in Windows 98. Using the C2 model in Rosetta1 (old version), the Rosetta3 hydraulic parameters were manually added to generate K0 and L. The dataset where the soil properties (i.e., percent sand, silt, clay, bulk density, and water contents at -33 and -1500 kPa) were taken from MesoSoilv2.0 The codes to generate the Rosetta3 parameters are in K_MesoSoilv2.0.ipynb.

In [4]:
# Checking the system version used
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In [ ]:
import os
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/GW_project/GWP/')

In [10]:
import numpy as np
import pandas as pd
from scipy.io import loadmat    # handles matlab files
import datetime

In [ ]:
# Loading the csv file containing the hydraulic parameters, K0, and L.
df = pd.read_csv('/content/drive/MyDrive/Drainage_estimate/Rosetta3_with_K0_and_L.csv', skiprows=1)

# Extracting every column in the file.
Site_ID = df['Site_ID']
theta_r = df['theta_r[cm3/cm3]']
theta_s = df['theta_s[cm3/cm3]']
alpha = df['alpha[1/kPa]']
n = df['n']
m = df['m']
Ks = df['Ks[cm/day]']
K0 = df['K0[cm/day]']
L = df['L']
L_d = 0.5    # default value of L

# Dictionaries to store data
df = {}
time = {}
tr60 = {}
matric_potential = {}
vwc = {}
Se = {}
K = {}        # unsaturated K with K0 and L = - values
K_1 = {}      # unsaturated K with K0 = Ks and L = 0.5

# Constants for Schneider et al. (2003) mp equation
mp_min = -2083
a = 3.35
deltaT_inf = 3.17

# Loading the matlab files
mat_folder = '/content/drive/MyDrive/Drainage_estimate/daily_deltaT'

# Putting all the matlab files in the directory while sorting them alphabetically
mat_files = sorted([f for f in os.listdir(mat_folder) if f.endswith('.mat')])

# Loading each file into a DataFrame and storing it with its site_id as the key
for file in mat_files:
  full_key = os.path.splitext(file)[0]       # splits the file name into two components (name and the extension)
  site_id = full_key.replace("deltaT", "")   # removes the extension and replaces it with nothing
  file_path = os.path.join(mat_folder, file)
  data = loadmat(file_path)
  df[site_id] = pd.DataFrame(data[full_key])

  # Extracting time and tr60 data
  time_raw = df[site_id].iloc[2:, 0].astype(float)     # starts at the 3rd row in the 1st column
  time[site_id] = pd.to_datetime([datetime.datetime.fromordinal(int(x)) + datetime.timedelta(days=x%1) - datetime.timedelta(days=366) for x in time_raw])     # converts the serial date numbers into datetime dates
  tr60[site_id] = df[site_id].iloc[2:, 4].astype(float)     # values start at the 3rd row in the 5th column

  # Daily matric potential [-, kPa]
  matric_potential[site_id] = mp_min / (1 + np.exp(-a * (tr60[site_id] - deltaT_inf)))

  # Matching the Site_ID from the csv file with the site_id in the matlab files
  # Removes any white spaces or hidden characters
  clean_Site_ID = Site_ID.str.strip()
  clean_site_id = site_id.strip()

  # Indexing to get the values for each column
  idx = clean_Site_ID[clean_Site_ID == clean_site_id].index

  # Checking and skipping the site_id not found in the csv file containing the parameters
  if idx.empty:
    #print(f"⚠️ Site ID '{site_id}' not found in Rosetta3 CSV. Skipping.")
    continue

  i = idx[0]
  theta_r_i = theta_r[i]
  theta_s_i = theta_s[i]
  alpha_i = alpha[i]
  n_i = n[i]
  m_i = m[i]
  Ks_i = Ks[i]
  K0_i = K0[i]
  L_i = L[i]

  # Volumetric water content [cm3/cm3]
  base = 1 + (-alpha_i * matric_potential[site_id])**n_i
  vwc[site_id] = theta_r_i + (theta_s_i - theta_r_i) * base**(-m_i)

  # Effective saturation (Se)
  Se[site_id] = (vwc[site_id] - theta_r_i) / (theta_s_i - theta_r_i)

  # Unsaturated hydraulic conductivity [cm/day] with K0 and L (-)
  term = (1 - Se[site_id]**(1/m_i))
  K[site_id] = K0_i * Se[site_id]**L_i * (1 - term**m_i)**2

  # Unsaturated hydraulic conductivity [cm/day] with K0 = Ks and L = 0.5
  term_1 = (1 - Se[site_id]**(1/m_i))
  K_1[site_id] = Ks_i * Se[site_id]**L_d * (1 - term**m_i)**2

In [ ]:
# Calculating the average daily volumetric water content [cm3/cm3]

vwc_avg = {}     # dictionary that stores average vwc data

# Calculating the average vwc for each site
for site_id in vwc:
    vwc_avg[site_id] = np.mean(vwc[site_id])

# Printing the average daily vwc per site
print(f'Total number of sites: {len(vwc_avg)}')   # shows the total number of sites
for site_id in sorted(vwc_avg):
    print(f'📍{site_id} → {vwc_avg[site_id]:.3f}')

"""
# Printing only the sites with NAN values
nan_count = 0
for site_id in sorted(vwc_avg):
    if np.isnan(vwc_avg[site_id]):
        print(f'⚠️ {site_id} → NaN')
        nan_count += 1
print(f'\nTotal number of sites with NaN vwc: {nan_count}')  # shows the total number of sites with NAN values
"""

In [37]:
# Calculating the average annual unsaturated hydraulic conductivity [mm/year]

# # dictionary that stores average K data
K_average = {}
K_1_average = {}

# Calculating the average K for each site
for site_id in K:
    K_average[site_id] = np.mean(K[site_id]) * 3650       # converting cm/day to mm/yr
    K_1_average[site_id] = np.mean(K_1[site_id]) * 3650   # converting cm/day to mm/yr

# Printing the average annual K per site
print(f'Total number of sites: {len(K_average)}')   # shows the total number of sites
print('---K values using K0 and the negative values of L---')
for site_id in sorted(K_average):
    print(f'📍{site_id} → {K_average[site_id]:.3f}')

print('---------------------------------------')

print('---K values using K0 = Ks and L = 0.5---')
print(f'Total number of sites: {len(K_1_average)}')   # shows the total number of sites
for site_id in sorted(K_1_average):
    print(f'📍{site_id} → {K_1_average[site_id]:.3f}')

Total number of sites: 129
---K values using K0 and the negative values of L---
📍ACME → 100.535
📍ADAX → nan
📍ALTU → 17.395
📍ALV2 → nan
📍ANT2 → 207.267
📍ANTL → 139.574
📍APAC → 85.414
📍ARD2 → 49.599
📍ARNE → 10.118
📍BEAV → 12.369
📍BESS → nan
📍BIXB → 366.099
📍BLAC → 108.375
📍BOIS → 9.602
📍BOWL → 106.495
📍BREC → 30.985
📍BRIS → 373.385
📍BROK → nan
📍BUFF → 28.699
📍BURB → nan
📍BURN → 38.195
📍BUTL → 37.889
📍BYAR → 90.108
📍CAMA → nan
📍CARL → 106.187
📍CENT → 260.746
📍CHAN → nan
📍CHER → 6.942
📍CHEY → 37.777
📍CHIC → nan
📍CLAR → nan
📍CLAY → nan
📍CLOU → nan
📍COOK → nan
📍COPA → 254.484
📍DURA → 35.892
📍ELKC → 79.507
📍ELRE → 70.270
📍ERIC → 24.955
📍EUFA → 211.437
📍EVAX → 7.588
📍FAIR → 86.707
📍FITT → 61.349
📍FORA → 363.260
📍FREE → 13.942
📍FTCB → 42.573
📍GOOD → 11.507
📍GRA2 → 97.830
📍GUTH → nan
📍HASK → 393.804
📍HECT → 100.163
📍HINT → 18.964
📍HOBA → 31.157
📍HOLD → 165.987
📍HOLL → 33.399
📍HOOK → 14.117
📍HUGO → nan
📍IDAB → nan
📍INOL → 128.285
📍JAYX → nan
📍KENT → nan
📍KETC → 60.560
📍KIN2 → 63.774
📍LAHO → 82.65